<a href="https://colab.research.google.com/github/Raian-Rahman/Pytorch-Practice/blob/main/(Residual_CNN_Practice)_Cifer10_Dataset_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import random_split,DataLoader

%matplotlib inline


### Loading the dataset

In [ ]:
train_data = datasets.CIFAR10('data',train=True,download='True',transform=transforms.ToTensor())
print(len(train_data))
train,val = random_split(train_data,[45000,5000])
train_loader = DataLoader(train,batch_size = 32)
val_loader = DataLoader(val,batch_size=32)

Extracting data/cifar-10-python.tar.gz to data
50000


# Defining the CNN Architecture

As it is a custom CNN so I am defining the architecture like below

Input -> CONV+MaxPOOL (5) -> CONV+MaxPOOL (5) -> CONV+MaxPOOL (5) -> CONV+MaxPOOL (3) -> FC -> FC -> Softmax

Initially the image size is 32 by 32

32x32x3 -> 28x28x8 -> 14x14x8 -> 10x10x16 -> 5x5x16 -> 4x4x32 -> 2x2x32

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()

        #convolutional layer
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size= 5)
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size= 5)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size= 2)

        #max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)


        #fully connected layer
        self.fc1 = nn.Linear(400,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,32)
        self.fc4 = nn.Linear(32,32)
        self.fc5 = nn.Linear(32,16)
        self.fc6 = nn.Linear(16,10)
        self.do  = nn.Dropout(p=0.3)
    def forward(self,x):
        #conv with ReLU and Pooling 
        x = self.conv0(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv1(x)
        
        x = F.relu(x)
        x = self.pool(x)

        # x = self.conv2(x)
        # x = F.relu(x)
        # x = self.pool(x)
        #flattenned layer        
        x = x.view(-1,5*5*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x1 = x = F.relu(self.fc3(x))
        x  = F.relu(self.fc4(x))
        x  = self.do(x)
        x  = F.relu(self.fc5(x+x1))
        x  = self.do(x)
        x  = F.relu(self.fc6(x))
        return x
        
model = Net().cuda()
print(model)

Net(
  (conv0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv1): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3072, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=16, bias=True)
  (fc6): Linear(in_features=16, out_features=10, bias=True)
  (do): Dropout(p=0.3, inplace=False)
)


# Defining a Loss Function and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(),lr=0.03)
loss = nn.CrossEntropyLoss()

In [ ]:
# define training loop

nb_epochs = 40

for epoch in range(nb_epochs):
    losses1 = []
    model.train()
    for batch in train_loader:
        x ,y = batch
        # x : batch_size * 1_channel * 28 * 28
        #b = x.size(0)
        #x = x.view(b,-1).cuda()
        #forward pass
        x = x.cuda()
        l = model(x) # l stands for logit
        
        #compute the loss fucnction
        J = loss(l,y.cuda())
        
        #cleaning the gradient 
        optimiser.zero_grad()
        #params.grad._zero()
       
        #compute accumulatinfg the partial gradient 
        J.backward() #params.
        #params.grad.add_(dJ/dparams)
        
        #update the parameters
        optimiser.step()
        #with torch.no_grad(): params = params - alpha*params.grad
        losses1.append(J.item())
    losses = []
    accuracies = []
    model.eval()
    for batch in val_loader:
        x ,y = batch
        # x : batch_size * 1_channel * 28 * 28
        x = x.cuda()
        #forward pass
        with torch.no_grad():
            l = model(x) # l stands for logit
            #compute the loss fucnction
            J = loss(l,y.cuda())
            losses.append(J.item())
            accuracies.append(y.cuda().eq(l.detach().argmax(dim=1)).float().mean())
    print(f'Epoch {epoch+1},train loss: {torch.tensor(losses1).mean():.2f}  validation loss: {torch.tensor(losses).mean():.2f}')
    print(f'Epoch {epoch+1},Accuracy: {torch.tensor(accuracies).mean():.5f}')

Epoch 1,train loss: 1.29  validation loss: 1.49
Epoch 1,Accuracy: 0.49025
Epoch 2,train loss: 1.28  validation loss: 1.51
Epoch 2,Accuracy: 0.48248
Epoch 3,train loss: 1.28  validation loss: 1.47
Epoch 3,Accuracy: 0.49940
Epoch 4,train loss: 1.27  validation loss: 1.54
Epoch 4,Accuracy: 0.47970
Epoch 5,train loss: 1.26  validation loss: 1.52
Epoch 5,Accuracy: 0.48507
Epoch 6,train loss: 1.26  validation loss: 1.58
Epoch 6,Accuracy: 0.45920
Epoch 7,train loss: 1.25  validation loss: 1.58
Epoch 7,Accuracy: 0.47114
Epoch 8,train loss: 1.25  validation loss: 1.53
Epoch 8,Accuracy: 0.47293
Epoch 9,train loss: 1.24  validation loss: 1.54
Epoch 9,Accuracy: 0.48209
Epoch 10,train loss: 1.24  validation loss: 1.53
Epoch 10,Accuracy: 0.47950
Epoch 11,train loss: 1.23  validation loss: 1.51
Epoch 11,Accuracy: 0.48666
Epoch 12,train loss: 1.23  validation loss: 1.53
Epoch 12,Accuracy: 0.48726
Epoch 13,train loss: 1.22  validation loss: 1.54
Epoch 13,Accuracy: 0.48786
Epoch 14,train loss: 1.21  val

```
Epoch 1,train loss: 2.30  validation loss: 2.30
Epoch 1,Accuracy: 0.11525
Epoch 2,train loss: 2.30  validation loss: 2.29
Epoch 2,Accuracy: 0.14291
Epoch 3,train loss: 2.24  validation loss: 2.16
Epoch 3,Accuracy: 0.22552
Epoch 4,train loss: 2.08  validation loss: 2.00
Epoch 4,Accuracy: 0.26931
Epoch 5,train loss: 1.93  validation loss: 2.17
Epoch 5,Accuracy: 0.23865
Epoch 6,train loss: 1.82  validation loss: 1.96
Epoch 6,Accuracy: 0.30912
Epoch 7,train loss: 1.72  validation loss: 1.86
Epoch 7,Accuracy: 0.36704
Epoch 8,train loss: 1.62  validation loss: 1.73
Epoch 8,Accuracy: 0.42456
Epoch 9,train loss: 1.56  validation loss: 1.61
Epoch 9,Accuracy: 0.46377
Epoch 10,train loss: 1.50  validation loss: 1.60
Epoch 10,Accuracy: 0.47452
Epoch 11,train loss: 1.46  validation loss: 1.54
Epoch 11,Accuracy: 0.49303
Epoch 12,train loss: 1.42  validation loss: 1.50
Epoch 12,Accuracy: 0.51791
Epoch 13,train loss: 1.38  validation loss: 1.50
Epoch 13,Accuracy: 0.51990
Epoch 14,train loss: 1.35  validation loss: 1.50
Epoch 14,Accuracy: 0.52189
Epoch 15,train loss: 1.32  validation loss: 1.50
Epoch 15,Accuracy: 0.52428
Epoch 16,train loss: 1.29  validation loss: 1.66
Epoch 16,Accuracy: 0.46477
Epoch 17,train loss: 1.23  validation loss: 1.47
Epoch 17,Accuracy: 0.53145
Epoch 18,train loss: 1.16  validation loss: 1.48
Epoch 18,Accuracy: 0.52727
Epoch 19,train loss: 1.13  validation loss: 1.47
Epoch 19,Accuracy: 0.53662
Epoch 20,train loss: 1.10  validation loss: 1.51
Epoch 20,Accuracy: 0.52946
Epoch 21,train loss: 1.08  validation loss: 1.49
Epoch 21,Accuracy: 0.52866
Epoch 22,train loss: 1.05  validation loss: 1.50
Epoch 22,Accuracy: 0.53185
Epoch 23,train loss: 1.03  validation loss: 1.54
Epoch 23,Accuracy: 0.53702
Epoch 24,train loss: 1.01  validation loss: 1.56
Epoch 24,Accuracy: 0.53304
Epoch 25,train loss: 0.99  validation loss: 1.62
Epoch 25,Accuracy: 0.52687
Epoch 26,train loss: 0.97  validation loss: 1.64
Epoch 26,Accuracy: 0.53025
Epoch 27,train loss: 0.95  validation loss: 1.62
Epoch 27,Accuracy: 0.54061
Epoch 28,train loss: 0.94  validation loss: 1.68
Epoch 28,Accuracy: 0.52767
Epoch 29,train loss: 0.92  validation loss: 1.65
Epoch 29,Accuracy: 0.52767
Epoch 30,train loss: 0.91  validation loss: 1.68
Epoch 30,Accuracy: 0.53105
Epoch 31,train loss: 0.89  validation loss: 1.73
Epoch 31,Accuracy: 0.52408
Epoch 32,train loss: 0.87  validation loss: 1.74
Epoch 32,Accuracy: 0.52767
Epoch 33,train loss: 0.86  validation loss: 1.72
Epoch 33,Accuracy: 0.53682
Epoch 34,train loss: 0.85  validation loss: 1.79
Epoch 34,Accuracy: 0.52588
Epoch 35,train loss: 0.84  validation loss: 1.76
Epoch 35,Accuracy: 0.53145
Epoch 36,train loss: 0.75  validation loss: 1.79
Epoch 36,Accuracy: 0.55792
Epoch 37,train loss: 0.70  validation loss: 1.82
Epoch 37,Accuracy: 0.54558
Epoch 38,train loss: 0.67  validation loss: 1.73
Epoch 38,Accuracy: 0.55852
Epoch 39,train loss: 0.66  validation loss: 1.79
Epoch 39,Accuracy: 0.55872
Epoch 40,train loss: 0.64  validation loss: 1.80
Epoch 40,Accuracy: 0.55613
```